

Group nr:  

Name 1 and CID: name surname (CID)

Name 2 and CID: name surname (CID)

In [ ]:
import numpy as np
import pygame
from IPython.display import Image
from BalanceRobot import BalanceRobot

## Introduction to Jupyter Notebook

Jupyter Notebook is a web application that allows you to write code, text and equations in one file. Each code cell can be run individually, but often depend on variables defined in cells above it. This means when you first open the file you will need to run the code cells starting from the top.

The text boxes are using Markdown format and to write the equations it uses LaTex format which can be initialized with \\$ on each side of the equation, e.g. $ax + b = c$. To edit a text box you double-click them and then to get back the normal text you can run the text box. To run a cell either use the play button on the top or ctrl enter. 



## Balance robot

In this assignment, we will perform parameter estimation on a balancing robot. First, we will show how the equations of motion can be derived, some of the parameters in these equations will be unknown to us and linear regression will be used to estimate them. The linear regression method requires data to be collected, which will be collected while the robot is balancing with a control algorithm. Below we can see a drawing of the robot.


<img src="img/BallanceRobot.png" width="300"/>

| Parameter | Description | To be estimated |
| --- | --- | --- |
| $m_1$ | Mass of main body [kg]| Yes |
| $l$ | Distance from wheel hub to $m_1$ [m] | Yes |
| $I_1$ | Moment of inertia for main body [kg $\textrm{m}^2$]| Yes |
| $m_2$ | Mass of wheels [kg]| No |
| $r$ | Radious of wheels [m]| No |
| $I_2$ | Moment of inertia for of wheels [kg $\textrm{m}^2$]| No |

| State | Description | 
| --- | --- |
| $\alpha_1$ | Angle of main body [rad]| 
| $\alpha_2$ | Angle of wheel from position 0 [rad]|
| $x_1$ | Position of $m_1$ in x-axis for main body [m]|
| $y_1$ | Position of $m_1$ in y-axis for main body [m]|
| $x_2$ | Position of $m_2$ in x-axis for wheel [m]|



## Equations of motion

Here we show one way to calculate the equations of motion for the system. To do this we will use the Euler-Lagrange method which builds on finding the stationary solution between the kinetic and potential energy of the system. 

We start with setting up the kinematic relationships. We define $(x_1, y_1)$ to be the position for the centre of mass of the main body and $x_2$ the position of the wheels, both with respect to the coordinate frame. The kinematics equations for the system then become:

$x_2 = - \alpha_2 r$

$x_1 = - \alpha_2 r - l sin(\alpha_1)$

$y_1 = r + l cos(\alpha_1)$

$\dot{x}_1 = - r \dot{\alpha}_2 - l cos(\alpha_1) \dot{\alpha_1}$

$\dot{y}_1 = -l sin(\alpha_1) \dot{\alpha}_1$

$\dot{x}_2 = - r \dot{\alpha}_2 $ 

Potential Energy for the system:

$V = l cos(\alpha_1)m_1g$

Kinematic energy:

$T = \frac{1}{2} m_1 (\dot{x}^2_1 + \dot{y}^2_1) + \frac{1}{2} m_2 \dot{x}^2_2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2$

From these the equations of motion for the balance robot can be calculated, here we use the Euler Lagrange method. First the Lagrangian is calculated by: 

$L = T - V$

and becomes:

$L = \frac{1}{2} m_1 (r^2 \dot{\alpha}^2_2 + 2 r \dot{\alpha}_2 l cos(\alpha_1) \dot{\alpha_1} + l^2  \dot{\alpha}_1^2) +
\frac{1}{2} m_2 r^2 \dot{\alpha}_2^2 + \frac{1}{2} I_1 \dot{\alpha}^2_1 + \frac{1}{2} I_2 \dot{\alpha}_2^2 -l cos(\alpha_1)m_1g$

Euler Lagrange equation is defined by: 

$\frac{d}{dt}(\frac{\partial L }{\partial \dot{q}_i}) - \frac{\partial L }{\partial q_i} = Q_i$

Where $q_1 = \alpha_1$, $q_2 = \alpha_2$, and $Q_i$ represent external force, in our case, the external force is the torque $M$ generated by the motors between the main body and the wheels. 

The equations of motion after simplification becomes:

$M = m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + m_1 l^2 \ddot{\alpha}_1 + I_1 \ddot{\alpha}_1  - lsin(\alpha_1)m_1g$

and

$-M =  m_1 r^2 \ddot{\alpha}_2 + m_1 rlcos(\alpha_1) \ddot{\alpha}_1 - m_1 rlsin(\alpha_1)\dot{\alpha}_1^2 + m_2 r^2 \ddot{\alpha}_2 + I_2 \ddot{\alpha}_2$

## Exersise 1:

To perform parameter identification with linear regression the equations need to be reformulated to the form $ \mathbf{y} = \mathbf{X}^T \mathbf{\theta}$, where $\mathbf{y}$ is a vector with 2 elements, $\mathbf{X}$ is a matrix and the vector $\mathbf{\theta}$ will be estimated using regression. There are 3 parameters we want to estimate, $m_1$, $l$ and $I_1$, all other parameters can be assumed to be known. Rewrite the two equations of motion to the form $ y = X^T \theta $ so that $m_1$, $l$ and $I_1$ can be extracted from $\theta$. 

Hint: It might not be possible to linearize the equations so that $\theta$ directly has the parameters $m_1$, $l$, $I_1$ by themselves, i.e. $\theta = [m_1, l, I_1]^T$, but instead it might contain combinations of the parameters from which we later can calculate the values of $m_1$, $l$, $I_1$. 



Answer:

$\left[ \begin{array}{c}
 \\
\end{array} \right] = 
\left[ \begin{array}{ccc}
  &  &  \\
   &  &   \end{array} \right]
\left[ \begin{array}{c}
 \\
  \\
\end{array} \right]$


## Simulator

For those interested, the equations of motion are not only interesting for parameter identification but can also be used to build a simulator for the system. The equations of motion are rewritten to a form where the state of the robot is easier to integrate.

$ \ddot{\alpha}_2   = \frac{-M -m_1 rlcos(\alpha_1) (M + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1) + m_1 rlsin(\alpha_1)\dot{\alpha}_1^2}{(m_1 r^2  - m_1^2 r^2 l^2 cos^2(\alpha_1)/ (m_1 l^2 + I_1) + m_2 r^2 + I_2)} $

$\ddot{\alpha}_1  = (M -  m_1 r \ddot{\alpha}_2 l cos(\alpha_1) + lm_1gsin(\alpha_1)) / (m_1 l^2 + I_1)$

From these equations, we can simulate the robot by either integrating accelerations with a fixed time step to get the velocity and positions or we can use a differential solver e.g. runge-kutta 45.  

## Controller
The robot is essentially an inverted pendilum on wheels, without a controller it will fall. The controller which can be seen below keeps the robot upright and moves it towords a referrence position on the x-axis. In the update function, there is a $\textit{pos_ref}$ variable that determines the reference signal for the controller, and will later be modified.  

<img src="img/Controller.png" width="500"/>

In [ ]:
class Controller(object):
    def __init__(self, dt, init_states):
        self.dt = dt
        # controller parameter for position controller
        self.K_pos = 3
        self.D_pos = 0.5
        # controller parameter for angle controller
        self.K_ang = 10
        self.I_ang = 1
        self.D_ang = 1
        # limit the maximum allowed reference angle
        self.max_angle = 0.8
        # variables for the integrator in the PID controller
        self.angle_int = 0
    
    def update(self, t, states): 
        # The update function will be called on every timestep and return the torque for the motors. 
        
        # The position reference to follow
        # ----------------------------------------------------------------
        pos_ref =  0.2*np.sin(t*2)
        # ----------------------------------------------------------------
        
        # PD controller for position
        err = self.K_pos*(pos_ref - states['pos']) - self.D_pos * states['vel']
        # The error is used as a reference angle, with upper and lower bounds. 
        ref_ang = -np.clip(err, -self.max_angle, self.max_angle)
        self.angle_int += self.dt * (ref_ang - states['a1'])
        # PID controller for the angle, generates the desired motor torque. 
        M = self.K_ang*(ref_ang-states['a1']) + self.I_ang*self.angle_int - self.D_ang*states['da1']
    
        return M


## Data collection

We need to collect data before we can do any regression on the problem. The data is collected when the robot is following a position reference signal. In the code below the data is collected in vectors, e.g. $\textrm{da1} = [\dot{\alpha}_1(t_0), \dot{\alpha}_1(t_1), ... , \dot{\alpha}_1(t_n)]$ and so on. The known parameters are g, r, m2 and I2. 

There are two variables you can change:

scale_noise: which scales the added noise to the measurements. 

sim_speed: which allows you to run the simulation faster then realtime. 


In [ ]:
## scale_noise, keep at 1 until later, change it later in exercise 7. 
# It controls the amount of noise applied to the measurements

# ----------------------------------------------------------------
scale_noise = 1
# ----------------------------------------------------------------

## Simulation speed, increase it for faster than real-time simulation.
sim_speed = 1

# number of steps and step size to run the simulation/data collection
n_steps = 1000
step_time = 1e-2

# Initiate the robot
robot = BalanceRobot(dt=step_time, sim_speed=sim_speed)
try:
    # True parameters in the robot.
    g = robot.p["g"] # gravity.
    r = robot.p["r"] # radius of wheel.
    m2 = robot.p["m2"] # mass of wheel.
    I2 = robot.p["I2"] # moment of intertia of wheel.

    # Initate the controller 
    state = robot.get_state()
    controller = Controller(dt=step_time, init_states=state)

    da1 = np.zeros(n_steps)
    a1 = np.zeros(n_steps)
    da2 = np.zeros(n_steps)
    a2 = np.zeros(n_steps)
    dda1 = np.zeros(n_steps)
    dda2 = np.zeros(n_steps)
    M = np.zeros(n_steps)

    ## Main loop 

    for i in range(n_steps):
        # calculate time
        t = i*step_time
        # calculate torque 
        state = robot.get_state()
        M_ = controller.update(t, state)
        # step one time step for robot simulation
        dda1_, dda2_ = robot.step(torque=M_)

        ## Add noice to data
        da1[i] = state['da1'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 1
        a1[i] = state['a1'] + np.random.normal(0.0, scale_noise*0.03) # alpha 1
        da2[i] = state['da2'] + np.random.normal(0.0, scale_noise*0.02) # d/dt alpha 2
        a2[i] = state['a2'] + np.random.normal(0.0, scale_noise*0.03) # alpha 2
        dda1[i] = dda1_ + np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 1
        dda2[i] = dda2_ + np.random.normal(0.0, scale_noise*0.01) # d^2/dt^2 alpha 2
        M[i] = M_ + np.random.normal(0.0, scale_noise*0.05) # torque 

    pygame.quit()
except:
    pygame.quit()


## Exercise 2:
For the first test, we set the position reference signal to be a sinusoidal shape, e.g. $\textit{pos_ref} = 0.2 \sin(2t)$. What would we expect to happen if we used a constant reference signal, i.e. $\textit{pos_ref} = 0$, instead? Which of the two reference signal would we expect to give better parameter estimation and why? 

Answer:



## Exercise 3: Linear regression 

Use linear regression to estimate the parameters $m_1$, $l$ and $I_1$. You have access to the vectors da1, da, da2, a2, dda1, dda2 and M, and you can use the variables g, r, m2 and I2.    

In [ ]:
#Answer: 

#est_m1 =
#est_l = 
#est_I1 =

print('Estimated mass = ', np.round(est_m1, 3), ' and true  = ', robot.p['m1'])
print('Estimated length = ', np.round(est_l, 3), ' and true value = ', robot.p['l'])
print('Estimated moment of inertia = ', np.round(est_I1, 3), ' and true value = ', robot.p['I1'])



## Exercise 4: Calculate the cost function 

When solving the linear regression problem we minimize a cost function $J(\theta) = \frac{1}{2 n} \sum_{i=1}^n \varepsilon_i^2$. Calculate the value of the cost function on the saved data with the parameters estimated. 

Hint: n = n_steps.

In [ ]:
#Answer: 

# cost_value = 

print('cost_value', cost_value)

## Exercise 5: Reference signal

Try to change the position reference signal in the controller, try a few different ones and report the identified parameters and loss value below together with the function used. You only need to change pos_ref in the update function in the controller. 

Hint: Try more complex functions by stacking sinusoidal, e.g. $\textit{pos_ref} = 0.2sin(2t) + 0.03cos(7t)$.


Answer:

| ref signal | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0 |  |  |  |  | 
| 0.2*np.sin(t*2) |  |  | |  | 
|  |  | | |  | 
| |  | | |  | 

## Exercise 6: Analyse cost function and reference signals. 

1. How do more complex reference signals affect parameter estimation?
2. Does the cost function value improve with better-identified parameters, why/why not?
3. Could you keep making the reference signal more and more complex for better estimation or is there a limit, why?


Answer:



 ## Exercise 7:  Noise

Explore how the noise levels added before the data collection affects the identification results and also the value of the cost function. Why is it that the cost function reacts differently compared to exercise 5? Use the position reference signal with the best parameter identification from exercise 5.

Answer:


$\textit{pos_ref} = $

| scale_noise | cost function value | $m_1$ | l| $I_1$ |
| --- | --- | --- | --- | --- |
| 0.1 |  | | |  | 
| 0.5 | |  | |  | 
| 1 |  |  | |  | 
| 2 |  | |  | | 
| 10 |  |  |  | | 
